In [1]:
import yaml
from random import shuffle

In [2]:
class ClassList(list):
    def __init__(self, arg):
        if type(arg) is str:
            arg = arg.split(' ')
        super().__init__(arg)

    def __repr__(self):
        return ' '.join(map(str, self))
    
    def add(self, cls):
        if cls not in self:
            self.append(cls)

    def remove(self, cls):
        if cls in self:
            super().remove(cls)

```js
function getFirstAncestorWithClass(element, className) {
    while (element.parentElement) {
        if (element.parentElement.classList.contains(className)) { break; }
        element = element.parentElement;
    }
    return element.parentElement; // will return null if exhausting the tree yeilds no element with the requested class
}

function rollUpScore(element) {
    let myScore = 0;
    let selected = 0;
    let answers = element.parentElement.children;
    
    for (let i = 0; i < answers.length; i++) {
        if (answers[i].classList.contains('selected')) {
            myScore += Number(answers[i].dataset.value);
            selected += 1;
        }
    }
    let questionContainer = getFirstAncestorWithClass(element, 'question-container');
    let delta = myScore - Number(questionContainer.dataset.value)
    questionContainer.dataset.value = myScore;
    questionContainer.dataset.answered = selected > 0;

    let quiz = getFirstAncestorWithClass(element, 'quiz');
    let quizScore = quiz.getElementsByClassName('quiz-score')[0];

    quizScore.dataset.value = Number(quizScore.dataset.value) + delta;
    quizScore.innerText = quizScore.dataset.value;
}
function is_all_ws(nod) {
    return !/[^\t\n\r ]/.test(nod.textContent);
}

function is_ignorable(nod) {
    return (
        nod.nodeType === 8 || // A comment node
        (nod.nodeType === 3 && is_all_ws(nod))
    ); // not a text node, all ws
}

function any_selected(classes) {
    let ans = false;

    for (cls in classes) {
        ans = ans || cls == 'selected';
    }
    return ans;
}

function selectAnswerExclusive(element) {
    // Remove 'selected' class from any previously selected answers
    let answers = element.parentElement.children; //
    //console.log(answers);
    
    for (let i = 0; i < answers.length; i++) {
        //console.log(answers[i].classList);
        answers[i].classList.remove('selected');
    }
    
    // Add 'selected' class to the clicked answer
    element.classList.add('selected');
    //console.log(element.parentElement);
    rollUpScore(element)
}

function selectAnswerMultiple(element) {
    // Remove 'selected' class from any previously selected answers
    element.classList.toggle('selected');
    rollUpScore(element)
}

function selectAnswerToggle(element) {
    // Remove 'selected' class from any previously selected answers
    element.classList.toggle('selected');
    rollUpScore(element)
}
````

In [ ]:
class Element():
    attrs = ['id', 'classes', 'data', 'callbacks']
    element_type = 'div'
    def __init__(self, content='', id='', classes=(), callbacks={}, data={}):
        self.callbacks = callbacks
        self.data = data
        self.content = content
        self.id = id
        self.classes = ClassList(classes)

    def repr_data(self):
        return ' '.join((f'data-{k.replace("_", "-")}="{v}"' for k, v in self.data.items()))
    def repr_callbacks(self):
        return ' '.join((f'{k}="{v}"' for k, v in self.callbacks.items()))

    def __repr__(self):
        attrstrs = [self.element_type]
        for attr in self.attrs:
            if getattr(self, attr):
                if hasattr(self, f'repr_{attr}'):
                    attrstrs.append(getattr(self, f'repr_{attr}')())
                else:
                    attrstrs.append(f'{attr if attr != "classes" else "class"}="{getattr(self, attr)}"')
            

        attrstr = ' '.join(attrstrs) 
        br = '\n'
        return f'<{attrstr}>{self.content if type(self.content) is str else (br + br.join(map(lambda o: str(o).rstrip(), self.content)) + br)}</{self.element_type}>\n'
    
class AnswerContainer(Element):
    def __init__(self, choices):
        super().__init__([_])

class Question(Element):
    def __init__(self, question, limit=None):
        if limit:
            data = {'limit': limit}
        else:
            data = {}
        super().__init__(content=question, data=data)


class Choice(Element):
    def __init__(self, answer, value):
        super().__init__(content=answer, data={'value': value})

class MultipleChoice(Element):
    def __init__(self, question, choices=(), correct=None, values=()):
        assert choices and type(choices) is not str and hasattr(choices, '__iter__'), "Argument `choices` must be an iterable of json serializable objects"

        if all((((type(c) is not str) and hasattr(c, '__iter__') and (len(c) == 2) and (type(c[1]) in {int, float})) for c in choices)):
            pass
        elif not values and correct:
            choices = [(c, int(c == correct)) for c in choices]
        elif values and not correct:
            choices = list(zip(choices, values))

        super().__init__(content=[question] + [Choice(*cv) for cv in choices], classes="question")

IndentationError: expected an indented block (<ipython-input-3-9fb9dcf66ed9>, line 33)

In [67]:
answer_set = []
correct = 'Topeka'
for choice in ['Topeka', 'Salina', 'Whichita', 'Kansas City']:
    answer_set.append(Element(content=choice,
                              classes='answer single-choice',
                              data={'value': int(choice == correct)},
                              callbacks={'onclick': 'selectAnswerExclusive(this)'})
                     )
shuffle(answer_set)



In [68]:
q = Element(classes='question-container', data={"answered": False, "value": 0},
            content=[Element(classes='question', content='What is the capital of Kansas?'),
                     Element(classes='answer-container', content=answer_set)
            ])

In [69]:
print(q)

<div class="question-container" data-answered="False" data-value="0">
<div class="question">What is the capital of Kansas?</div>
<div class="answer-container">
<div class="answer single-choice" data-value="1" onclick="selectAnswerExclusive(this)">Topeka</div>
<div class="answer single-choice" data-value="0" onclick="selectAnswerExclusive(this)">Kansas City</div>
<div class="answer single-choice" data-value="0" onclick="selectAnswerExclusive(this)">Whichita</div>
<div class="answer single-choice" data-value="0" onclick="selectAnswerExclusive(this)">Salina</div>
</div>
</div>

